In [1]:
import pandas as pd
from pathlib import Path

path = Path("datasets/ridership/CTA_-_Ridership_-_Daily_Boarding_Totals.csv")
df = pd.read_csv(path, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"]  # shorter names
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)  # no need for total, it's just bus + rail
df = df.drop_duplicates()  # remove duplicated months (2011-10 and 2014-07)

df.head()

,day_type,bus,rail
date,,,
2001-01-01,U,297192,126455
2001-01-02,W,780827,501952
2001-01-03,W,824923,536432
2001-01-04,W,870021,550011
2001-01-05,W,890426,557917


#### Let's see how fit a SARIMA model to  the rail time series, will use it to forecast tomorrow's ridership. Will pretend today is the last day of May 2019, and want to predict for tomorrow's ridership (June 1, 2019) 

In [6]:
from statsmodels.tsa.arima.model import ARIMA

origin, today = "2019-01-01", "2019-05-31"
rail_series = df.loc[origin:today]["rail"].asfreq("D")
rail_series

date
2019-01-01    245852
2019-01-02    573542
2019-01-03    627781
2019-01-04    628514
2019-01-05    348257
               ...  
2019-05-27    256757
2019-05-28    694292
2019-05-29    717681
2019-05-30    735508
2019-05-31    738322
Freq: D, Name: rail, Length: 151, dtype: int64

In [13]:
model = ARIMA(rail_series, order=(1,0,0), seasonal_order=(0,1,1,7))
fitted_model = model.fit()

In [15]:
y_pred = fitted_model.forecast()
y_pred

2019-06-01    427758.626286
Freq: D, dtype: float64

#### Code explanation:
    1. Import ARIMA class
    2. Take a time series from January 1 to May 31 of 2019
    3. Here used frquency on a daily basis
    4. hyperparameters: order=(1,0,0) means p=1, q=0, d=0
    5. hyperparameters: seasonal_order=(0,1,1,7) means P=0, D=1, Q=1, s=7
    5. Next preict for June 1
    
The forecast is 427759 passengers, when in fact there were 379,044. 12.9% off-that's pretty bad, actually worse than native forecasting (426,932-off by 12.6%). Now, let's run the model in a loop, and compute MAE for the month March, April, and May.

In [18]:
origin, start_date, end_date = "2019-01-01", "2019-03-01", "2019-05-31"
time_period = pd.date_range(start_date, end_date)
rail_series = df.loc[origin:end_date]["rail"].asfreq("D")
y_preds = []

In [20]:
for today in time_period.shift(-1):
    model = ARIMA(rail_series[origin:today], order=(1,0,0), seasonal_order=(0,1,1,7))
    fitted_model = model.fit()
    y_pred = fitted_model.forecast()[0]
    y_preds.append(y_pred)

In [22]:
y_preds = pd.Series(y_preds, index=time_period)
mae = (y_preds - rail_series[time_period]).abs().mean()
mae

32040.720089838305

#### Now, it's better than the native forecasting where MAE was 42,143 although the SARIMA model is not perfect, but still it beats the native one. 

#### For the hyperparameters, we can apply grid search technique like brute force, p, q, P, Q are typically varies fro 0 to 2, sometimes 5 or 6, and for d and D it's 0 to 1, sometimes 2. For s, we use 7 because of there is a string weekly seasonaliy.